In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd

# 1.데이터 로드

In [2]:
data_path = '../data'
# data_path = '/Users/pervin0527/upstage-ml-regression-ml11/data'
train_path = f'{data_path}/new/new_train_ver2.csv'
test_path  = f'{data_path}/new/new_test_ver2.csv'

print(os.listdir(data_path))

['new', 'subway_feature.csv', 'sampled_data.csv', 'train.csv', 'test.csv', 'bus_feature.csv', 'sample_submission.csv', '.DS_Store']


In [3]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
print('Train data shape : ', train_data.shape, 'Test data shape : ', test_data.shape)

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)


## 1-1.병합

In [4]:
# train/test 구분을 위한 칼럼 생성.
train_data['is_test'] = 0
test_data['is_test'] = 1
data = pd.concat([train_data, test_data]) # 하나의 데이터로 합친다.

print(data.shape)

(1128094, 53)


# 2.컬럼 제거

In [5]:
## 결측치가 너무 많고, 대체할 가치가 없다고 생각함.
display(data['해제사유발생일'].value_counts())

해제사유발생일
20220211.0    151
20200727.0     43
20200714.0     40
20200720.0     37
20200715.0     32
             ... 
20200521.0      1
20201018.0      1
20200523.0      1
20200705.0      1
20200613.0      1
Name: count, Length: 1033, dtype: int64

In [6]:
## 공백이라는 의미없는 데이터가 너무 많음.
display(data['등기신청일자'].value_counts())

등기신청일자
            1111271
20230630        585
20230831        496
20230428        394
20230731        374
             ...   
20230105          3
20230125          3
20230109          1
20230102          1
20230104          1
Name: count, Length: 182, dtype: int64

In [7]:
## '-' 가 너무 많은 데이터로 존재.
display(data['거래유형'].value_counts())

거래유형
-       1086451
중개거래      38123
직거래        3520
Name: count, dtype: int64

In [8]:
## '-' 가 너무 많은 데이터로 존재.
display(data['중개사소재지'].value_counts())

중개사소재지
-                          1090013
서울 노원구                        2627
서울 송파구                        2540
서울 강남구                        2508
서울 강동구                        2038
                            ...   
부산 해운대구, 서울 강남구, 서울 용산구          1
경기 의정부시, 서울 영등포구                 1
서울 노원구, 서울 양천구                   1
광주 동구, 서울 노원구                    1
경기 연천군, 서울 송파구                   1
Name: count, Length: 739, dtype: int64

In [9]:
## 제거할컬럼
drop_columns = [
    # '해제사유발생일', '등기신청일자', '거래유형', 'k-사용검사일-사용승인일', 'k-등록일자', 'k-수정일자', '단지승인일', '단지신청일'
    '중개사소재지', 'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-홈페이지', 
    '고용보험관리번호', '기타/의무/임대/임의=1/2/3/4', '사용허가여부', '관리비 업로드'
]
data = data.drop(drop_columns, axis=1)
print(data.shape)

(1128094, 44)


In [10]:
## 결측치를 확인.
data.isnull().sum()

시군구                             0
번지                              0
본번                              0
부번                              0
아파트명                         2136
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1121899
등기신청일자                          0
거래유형                            0
k-단지분류(아파트,주상복합등등)         877273
k-세대타입(분양형태)               876125
k-관리방식                     876125
k-복도유형                     876454
k-난방방식                     876125
k-전체동수                     877207
k-전체세대수                    876125
k-건설사(시공사)                 877637
k-시행사                      877834
k-사용검사일-사용승인일              876259
k-연면적                      876125
k-주거전용면적                   876170
k-관리비부과면적                  876125
k-전용면적별세대현황(60㎡이하)         876170
k-전용면적별세대현황(60㎡~85㎡이하)     876170
k-85㎡~135㎡이하  

# 3.파생변수

## 3-0.전용면적 rename

In [11]:
data = data.rename(columns={'전용면적(㎡)':'전용면적'})

In [12]:
data.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', 'k-단지분류(아파트,주상복합등등)',
       'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
       'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적',
       'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)',
       'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-등록일자', 'k-수정일자', '경비비관리형태', '세대전기계약방법',
       '청소비관리형태', '건축면적', '주차대수', '단지승인일', '좌표X', '좌표Y', '단지신청일', 'target',
       'is_test'],
      dtype='object')

## 3-1."시군구" $\rightarrow$ "시", "군", "구"로 분할

In [13]:
# '시군구' 컬럼을 공백을 기준으로 분할
data[['시', '구', '동']] = data['시군구'].str.split(' ', expand=True)
data = data.drop(['시군구', '시', '번지'], axis=1)

data.columns

Index(['본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명',
       '해제사유발생일', '등기신청일자', '거래유형', 'k-단지분류(아파트,주상복합등등)', 'k-세대타입(분양형태)',
       'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)',
       'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적',
       'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하',
       'k-135㎡초과', 'k-등록일자', 'k-수정일자', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '단지승인일', '좌표X', '좌표Y', '단지신청일', 'target', 'is_test',
       '구', '동'],
      dtype='object')

In [14]:
print(len(data['구'].unique()))
print(data['구'].unique())

25
['강남구' '강동구' '강북구' '강서구' '관악구' '광진구' '구로구' '금천구' '노원구' '도봉구' '동대문구' '동작구'
 '마포구' '서대문구' '서초구' '성동구' '성북구' '송파구' '양천구' '영등포구' '용산구' '은평구' '종로구' '중구'
 '중랑구']


In [15]:
print(len(data['동'].unique()))
print(data['동'].unique())

337
['개포동' '논현동' '대치동' '도곡동' '삼성동' '세곡동' '수서동' '신사동' '역삼동' '압구정동' '일원동' '자곡동'
 '청담동' '강일동' '율현동' '고덕동' '길동' '둔촌동' '명일동' '상일동' '성내동' '암사동' '천호동' '미아동'
 '번동' '가양동' '수유동' '우이동' '내발산동' '등촌동' '공항동' '마곡동' '방화동' '염창동' '화곡동' '남현동'
 '봉천동' '신림동' '광장동' '구의동' '자양동' '군자동' '화양동' '개봉동' '고척동' '중곡동' '가리봉동' '구로동'
 '오류동' '천왕동' '독산동' '온수동' '궁동' '신도림동' '항동' '가산동' '시흥동' '공릉동' '상계동' '월계동'
 '중계동' '하계동' '도봉동' '방학동' '쌍문동' '창동' '답십리동' '용두동' '이문동' '신설동' '장안동' '전농동'
 '회기동' '제기동' '청량리동' '휘경동' '노량진동' '대방동' '동작동' '본동' '사당동' '상도동' '상도1동'
 '신대방동' '흑석동' '공덕동' '노고산동' '대흥동' '도화동' '마포동' '상수동' '상암동' '서교동' '성산동'
 '신공덕동' '신수동' '신정동' '아현동' '동교동' '망원동' '염리동' '용강동' '중동' '창전동' '하중동' '합정동'
 '남가좌동' '냉천동' '북가좌동' '연남동' '토정동' '현석동' '대현동' '연희동' '영천동' '충정로3가' '현저동'
 '홍은동' '홍제동' '북아현동' '창천동' '천연동' '충정로2가' '합동' '내곡동' '반포동' '방배동' '서초동' '신원동'
 '우면동' '잠원동' '양재동' '금호동1가' '금호동2가' '금호동3가' '성수동1가' '성수동2가' '금호동4가' '송정동'
 '도선동' '마장동' '사근동' '상왕십리동' '옥수동' '하왕십리동' '행당동' '응봉동' '용답동' '홍익동' '길음동'
 '돈암동' '동소문동5가' '동소문동7가' '보문동3가' '동선동1가' '동선동3가' '동선동4가' 

## 3-2."계약년월", "계약일"을 "계약날짜"로 병합

In [16]:
data['계약날짜'] = pd.to_datetime(data['계약년월'].astype(str) + data['계약일'].astype(str), format='%Y%m%d')
data = data.drop(columns=['계약년월', '계약일'])
data.head(3)

,본번,부번,아파트명,전용면적,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,...,주차대수,단지승인일,좌표X,좌표Y,단지신청일,target,is_test,구,동,계약날짜
0,658.0,1.0,개포6차우성,79.97,3,1987,언주로 3,NaN,,-,...,262.0,2022-11-17 13:00:29.0,127.05721,37.476763,2022-11-17 10:19:06.0,124000.0,0,강남구,개포동,2017-12-08
1,658.0,1.0,개포6차우성,79.97,4,1987,언주로 3,NaN,,-,...,262.0,2022-11-17 13:00:29.0,127.05721,37.476763,2022-11-17 10:19:06.0,123500.0,0,강남구,개포동,2017-12-22
2,658.0,1.0,개포6차우성,54.98,5,1987,언주로 3,NaN,,-,...,262.0,2022-11-17 13:00:29.0,127.05721,37.476763,2022-11-17 10:19:06.0,91500.0,0,강남구,개포동,2017-12-28


# 4.범주형 변수 결측치 처리

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1128094 entries, 0 to 9271
Data columns (total 43 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   본번                      1128094 non-null  float64       
 1   부번                      1128094 non-null  float64       
 2   아파트명                    1125958 non-null  object        
 3   전용면적                    1128094 non-null  float64       
 4   층                       1128094 non-null  int64         
 5   건축년도                    1128094 non-null  int64         
 6   도로명                     1128094 non-null  object        
 7   해제사유발생일                 6195 non-null     float64       
 8   등기신청일자                  1128094 non-null  object        
 9   거래유형                    1128094 non-null  object        
 10  k-단지분류(아파트,주상복합등등)      250821 non-null   object        
 11  k-세대타입(분양형태)            251969 non-null   object        
 12  k-관리방식                

## 4-1.k-세대타입(분양형태)

- 개인이 분양인이지, 임대인지 계약 정보를 대체할 수 없기 때문에 ```nan```은 ```unknown```으로 처리한다. 
- 컬럼 이름을 ```분양형태```로 대체한다.
- 결측치를 채운다면 **모델 예측**을 예상하고 있다.

In [18]:
display(data['k-세대타입(분양형태)'].unique())

array(['분양', '기타', nan, '임대'], dtype=object)

In [19]:
data['k-세대타입(분양형태)'] = data['k-세대타입(분양형태)'].fillna('unknown')
data['k-세대타입(분양형태)'] = data['k-세대타입(분양형태)'].replace('기타', 'unknown')
data = data.rename(columns={'k-세대타입(분양형태)':'분양형태'})
display(data['분양형태'].value_counts())

분양형태
unknown    917763
분양         208098
임대           2233
Name: count, dtype: int64

## 4-2.k-단지분류(아파트,주상복합등등)

- 건물이 [아파트, 주상복합, 연립주택, 도시형 생활주택(아파트), 도시형 생활주택(주상복합), nan] 어떤 유형인지 나타냄.
- nan을 ```unknown```으로 대체
- 컬럼 이름을 ```부동산유형```으로 변경한다.

In [20]:
display(data['k-단지분류(아파트,주상복합등등)'].unique())

array(['아파트', '주상복합', nan, '연립주택', '도시형 생활주택(아파트)', '도시형 생활주택(주상복합)'],
      dtype=object)

In [21]:
data['k-단지분류(아파트,주상복합등등)'] = data['k-단지분류(아파트,주상복합등등)'].fillna('unknown')
data = data.rename(columns={'k-단지분류(아파트,주상복합등등)':'부동산유형'})

print(data['부동산유형'].value_counts())

부동산유형
unknown           877273
아파트               238586
주상복합               11450
도시형 생활주택(주상복합)       505
도시형 생활주택(아파트)        155
연립주택                 125
Name: count, dtype: int64


## 4-3.k-관리방식, k-복도유형, k-난방방식

In [22]:
display(data['k-관리방식'].unique())

array(['자치관리', '위탁관리', '직영', nan], dtype=object)

In [23]:
data['k-관리방식'] = data['k-관리방식'].fillna('unknown')
print(data['k-관리방식'].value_counts())

k-관리방식
unknown    876125
위탁관리       230361
자치관리        18144
직영           3464
Name: count, dtype: int64


In [24]:
display(data['k-복도유형'].unique())

array(['계단식', '혼합식', '복도식', '타워형', '기타', nan], dtype=object)

In [25]:
data['k-복도유형'] = data['k-복도유형'].fillna('unknown')
data['k-복도유형'] = data['k-복도유형'].replace('기타', 'unknown')
print(data['k-복도유형'].value_counts())

k-복도유형
unknown    877536
계단식        124964
혼합식         85513
복도식         38710
타워형          1371
Name: count, dtype: int64


In [26]:
display(data['k-난방방식'].unique())

array(['개별난방', '지역난방', '기타', nan, '중앙난방'], dtype=object)

In [27]:
data['k-난방방식'] = data['k-난방방식'].fillna('unknown')
data['k-난방방식'] = data['k-난방방식'].replace('기타', 'unknown')
print(data['k-난방방식'].value_counts())

k-난방방식
unknown    878216
개별난방       152365
지역난방        84826
중앙난방        12687
Name: count, dtype: int64


## 4-4.k-건설사(시공사) & k-시행사

- k-건설사(시공사) 는 ```k-건설사```로 이름 변경.
- k-시행사에는 '모름'이라는 데이터가 존재. 이를 nan과 함께 unknown으로 처리.

In [28]:
display(data['k-건설사(시공사)'].unique())

array(['우성건설', '박연정', '대한주택공사', '현대건설(주)', '동양고속건설주식회사', '대우건설',
       '대림산업(주)', '삼성물산', nan, '삼성물산주식회사', '현대,쌍용,GS', '쌍용건설(주)',
       '현대산업개발', '한보', '삼성중공업', '대성산업', '중앙건설', '세방기업', '태영건설', '신동아',
       '신동아건설', '고려개발(주)', '대우건설,대림산업,태영건설', '한라, 두진', '고려산업개발(주)',
       '삼성건설', '롯데건설', 'GS건설', '(주)한화건설', '한신공영', '(주)건영,삼익주택,라이프주택',
       '계룡건설산업(주)', '한양수자인', '건영종합건설(주)', '청담삼익아파트', '계룡건설(주)', '삼환기업',
       '한신공영(주)', 'KR산업', '동부건설', '고속도로관리공단', '동부건설(주)외1', 'SH공사',
       '남광, 대우', 'sh공사', '청광종합건설(주)', '중앙건설(주)', 'LG건설', '신동아건설(주)',
       '현대산업개발(주)', '(주)청구주택', '삼호건설', '(주)삼호', '한솔건설', '현대,대림',
       '신동아 건설(주)', '임광토건(주)', '롯데건설(주)', '벽산건설', '동부건설(주)', '현대 성우건설',
       '홍익종합건업(주)', '한양주택 (주)', '한진중공업(주)', '성지건설', '남광토건(주)', 'gs건설',
       '신명건설,신원건설,미주건설', 'KCC', '두산, 이수건설', '두산건설', 'TEC건설',
       '현대건설,현대산업개발,한진중공업', '성원건설(주)', '라인건설', '코오롱글로벌(주)', '금호건설',
       '(주)대보건설', '(주)한양', '경남기업', '에스에이치공사', '금호', '(주)서광건설산업',
       '우림건설(주)', '중앙하이츠', '청구', '우림건설', '삼환, 삼익건설

In [29]:
data["k-건설사(시공사)"] = data["k-건설사(시공사)"].fillna('unknown')
data = data.rename(columns={'k-건설사(시공사)':'k-건설사'})

In [30]:
synonyms = {
    '삼성물산': ['삼성물산주식회사', '삼성물산(주)', '삼성물산 건설부문', '삼성물산(주) 건설부문', '삼성물산,두산건설', '삼성물산(주) 주택부문', "삼성건설", "삼성건설(주)", "삼성래미안", "삼성물산", "(주)삼성물산", "주시회사 현대건설"],
    '현대건설': ['현대건설(주)', '현대건설주식회사', '(주)현대건설', '현대산업개발', '현대산업개발(주)', 'HDC현대산업개발', '현대bs&c', '현대BS&C', "(주)현대산업개발", "현대산업개발", "주식회사 현대건설", "현대산업개발 주", "현대종합건설"],
    '대림산업': ['대림산업(주)', '대림산업주식회사', '(주)대림산업', '대림건설', '대림건설(주)', "(주)대림", "대림", "대림산업", "대림산업 (주)"],
    '대우건설': ['대우건설(주)', '대우푸르지오', "(주)대우", "(주)대우건설", "대우건설", "주)대우건설", "대우경남"],
    'GS건설': ['GS건설(주)', '지에스 건설', '지에스건설주식회사', '지에스건설(주)', "(주)gs공사", "GS 건설", "GS건설", "GS건설주식회사", "GS자이 건설", "gs건설", "럭키외"],
    '한라': ['(주)한라', '한라건설', '(주)한라건설'],
    '롯데건설': ['롯데건설(주)', '롯데건설주식회사', "(주)롯데건설", "롯데건설"],
    '한화건설': ['(주)한화건설', '한화건설(주)'],
    '두산건설': ['두산건설주식회사', '두산건설(주)'],
    '우성건설': ['우성건설(주)', '(주)우성건설'],
    '포스코건설': ['(주)포스코건설', '포스코, 포스코A&C', '포스코'],
    'SH공사': ['sh공사', '에스에이치공사', 'SH', "SH공사", "sh 공사", "(주)에스에이치건설", "서울도시개발공사", "주택공사"],
    '건영': ['(주)건영', '(주) 건영', "(주) 건영", "건영", "건영건설", "건영종합건설(주)", " (주) 건영"],
    '신동아건설': ['신동아건설(주)', '신동아 건설(주)'],
    '계룡건설': ['계룡건설(주)', '계룡건설산업(주)'],
    '코오롱건설': ['코오롱글로벌(주)', '코오롱글로벌', '코오롱건설주식회사'],
    '성원건설': ['성원건설(주)', '(주)성원건설'],
    "금호": ["(주)금호", "(주)금호건설", "(주)금호산업", "금호", "금호건설", "금호건설(주)", "금호산업", "금호자이"],
    "동아건설": ["(주)동아건설산업", "동아건설", "동아건설산업(주)"],
    "동일토건": ["(주)동일토건", "동일토건"],
    "LG건설": ["엘지건설(주)"],

    "삼호건설": ["삼호건설", "삼호주택", "(주)삼호"],
    "서광산업": ["(주)서광건설산업"],
    "서희건설": ["(주)서희건설", "서희건설", "주식회사 서희건설"],
    "신성건설": ["(주)신성건설", "신성건설", "신성건설(주)"],
    "신창건설": ["(주)신창건설", "신창건설", "신창건설(주)->우방건설"],
    "쌍용건설": ["(주)쌍용건설", "쌍용건설", "쌍용건설(주)"],
    "우방건설": ["(주)우방", "우방", "우방건설"],
    "우리건설": ["(주)우리건설", "우리건설"],
    "일성건설": ["(주)일성건설", "일성건설", "(주)일성건설"],
    "임광건설": ["임광토건(주)"],
    "청구건설": ["청구", "청구주택"],

    "중앙건설": ["(주)중앙건설", "중앙건설", "중앙건설(주)"],
    "진흥기업": ["(주)진흥기업", "진흥기업", "진흥기업(주)"],
    "태영건설": ["(주)태영", "(주)태영건설", "태영건설"],
    "한양건설": ["(주)한양", "(주)한양건설", "한양", "한양건설", "한양", "한양건설", "한양수자인", "한양주택 (주)", "?양건설", "한양산업개발"],
    "한진중공업": ["(주)한진중공업", "한진중공업", "한진중공업(주)", "한신공영", "한신공영(주)"],
    "한진건설": ["한진건설(주)"],
    "효성건설": ["(주)효성", "(주)효성건설", "효성건설"],
    "KCC건설": ["KCC", "kcc건설"],
    "동부건설": ["동부건설", "동부건설(주)", "동부건설(주)외1"],
    "이수건설": ["이수건설", "이수건설(주)", "이수건설,대우엔지니어링"],
    "삼성중공업": ["삼성중공업", "삼성중공업(주)"],
    "호반건설" : ["(주)호반건설"],
    "LIG건설" : ["LIG건설", "LIG건설(주)"],
    "SK건설": ["(주)에스케이건설"],
    "경남기업": ["경남기업", "경남기업(주)"],
    "경향건설": ["경향건설(주)"],
    "계룡건설": ["계룡건설산업"],
    "고려개발": ["고려개발", "고려개발(주)", "고려산업개발", "고려산업개발(주)"],
    "극동건설" : ["극동건설", "극동건설(주)"],
    "나산건설" : ["나산종합건설"],
    "라이프건설": ["라이프주택"],
    "신도건설": ["(주)신도종합건설"],
    "쌍용건설": ["쌍용"],
    "우림건설": ["우림건설", "우림건설(주)"],
    "유원건설": ["유원건설", "유원건설(주)", '울트라건설', '울트라건설(주)', "유원울트라"],
    "월드건설": ['월드건설', '월드건설(주)'],
    '인정건설' : ['인정건설', '인정건설(주)'],
    "흥화건설": ["흥화공업", "흥화공업(주)"],
    "한솔건설": ["한솔"],
    "한보건설": ["한보", "한보철강"],
    "고려개발": ["고려개발 "],
    "동양메이저건설": ["동양메이저(주)"],
    "동일": ["동일토건", "동일하이빌"],
    "두산건설": ["두산", "두산중공업"],
    "미주실업": ["미주실업주식회사"],
    "벽산건설": ["벽산건설(주)"],
    "삼익건설": ["삼익주택(주)", "삼익주택", "삼익주택(주)", "삼익건설(주)"],
    "삼정건설": ["삼익건설(주)"],
    "삼익주택": ["삼익주택(주)"],
    "삼정건설": ["삼정기업"],
    "삼환": ["삼환기업"],
    "세양건설": ["세양건설산업"],
    "신동아건설": ['신동아'],
    "양우건설": ['양우건설(주)', "양우건설사"],
    "우성건설": ["우성"],
    "우정건설": ["우정건설 주식회사"],
    "중앙건설": ["중앙하이츠"],
    "진흥건설": ["진흥기업", "진흥연립재건축조합"],
    '풍림건설': ['풍림산업(주)', '풍림산업'],
    '한보건설': ['한보', '한보철상'],
    "(주)신일건업": ['신일산업'],
    "동일": ['동일토건'],
    '세양건설': ['세양건설산업(주)']
}

def replace_synonyms(value, synonyms_dict):
    for key, values in synonyms_dict.items():
        if value in values:
            return key
    return value

data['k-건설사'] = data['k-건설사'].apply(lambda x: replace_synonyms(x, synonyms))

company_list = sorted(data["k-건설사"].unique())
print(len(company_list))
for c in company_list:
    print(c)

193
 일신건영
(주) 신한건설
(주)건설알포메
(주)건영,삼익주택,라이프주택
(주)대림/ (주)삼성
(주)대보건설
(주)대우자동차판매
(주)도시공감
(주)삼전건설
(주)순영종합건설
(주)신일건업
(주)쌍용건설
(주)우리창우
(주)우성건설
(주)월드건설
(주)중앙건설
(주)청구주택
(주)현진종합건설
BYC
GS건설
JR건설
KCC건설
KR산업
LG건설
LIG건설
SH공사
SK건설
TEC건설
unknown
갑을건설
건영
경남기업
경동건설
경향건설
계룡건설
계룡건설(주)
계룡건설산업(주)
고려개발
고려개발(주)
고려산업개발
고려산업개발(주)
고속도로관리공단
공영토건
극동건설
금하건설
금호
나산건설
남광, 대우
남광토건(주)
남해건설
대동건설
대림,삼성,현대,현산
대림산업
대림산업, 롯데건설
대명종합건설
대백건설
대성산업
대우,럭키개발,롯데건설,선경건설,태평양건설
대우건설
대우건설,대림산업,태영건설
대주건설
대창건설
대한전척(주)
대한주택공사
대호아이피종합건설
도시개발공사
돈암정릉구역주택재개발정비사업조합
동부건설
동선구역주택재개발조합
동성개발
동아건설
동양고속건설주식회사
동양메이저건설
동원산업건설
동일
동일토건
동일하이빌
두산, 이수건설
두산건설
두산건설(주)
두산건설주식회사
라성건설
라이프건설
라인건설
롯데건설
롯데건설,우성건설
문영건설
미원건설(주)
미주실업
미주실업,동방티앤씨
박연정
벽산건설
보람건설(주)
삼부토건
삼성물산
삼성물산, 대림산업
삼성물산, 현대산업개발
삼성물산,한화건설
삼성중공업
삼익건설
삼정건설
삼중종합건설(주)
삼호건설
삼환
삼환, 삼익건설
상부종합건설(주)
서광산업
서초건설
서희건설
선경건설
성원건설
성지건설
세방기업
세양건설
신도건설
신동아 건설(주)
신동아건설
신동아건설(주)
신명건설,신원건설,미주건설
신성건설
신영건설
신창건설
신홍선건설
쌍용건설
쌍용건설(주)
아남건설주식회사
아키데코
양우건설
예성종합건설
우리건설
우림건설
우방건설
우성건설
우성건설(주)
우정건설
월드건설
유원, 선경, 대우건설
유원건설
이수건설
인덕주택재건축정비사업

In [31]:
display(data['k-시행사'].unique())

array(['모름', '우성건설', 'LG건설', '대한주택공사', '대우건설', '동양고속건설주식회사',
       '경복아파트주택재건축정비사업조합', '삼성물산', '신해청재건축조합', nan, '우성2차리모델링주택조합',
       '프리즘(주)', '도곡주공재건축조합', '도곡동신아파트리모델링주택조합', '을지조합', '대치도곡제2아파트재건축조합',
       '한보', '도곡연합주택조합', '대성산업', '상아3차아파트주택재건축정비사업조합', '중앙건설', 'SH공사',
       'sh공사', 'sh 공사', '한국토지주택공사', 'LH강남지역본부', '한라, 두진',
       '신사동 한영아파트 1지구 재건축조합', '삼성건설', '롯데건설', '영동1재건축조합',
       '개나리6차아파트주택재건축정비사업조합', '.', '(주)한화건설', '한신공영',
       '건영종합건설, 삼익주택,라이프주택', '청강연합주택조합', '청담삼익아파트', '현대산업개발',
       '청담 한양아파트 주택재건축사업조합', '현대건설(주)', '에스에이치공사', '서울주택도시공사', '남광토건, 대우',
       '청광종합건설(주)', '길동진흥아파트주택재건축정비사업조합', '청수연립재건축조합', '명일시영아파트 재건축조합',
       '고덕주공4단지주택재건축정비사업조합', '둔촌역지역주택조합', '삼호건설', '대농연립재건축조합', '샘하우징',
       '현대,대림', '(주)포유', '(주)디씨엠하우징', '생보부동산신탁', '미아제4구역재개발정비사업조합',
       '미아제10구역주택재개발조합', '미아1-2재개발조합', '미아10-1구역재개발조합', '양지연립주택조함',
       '건정종합건축사사무소', '번동한양아파트주택조합', '대한주택보증(주)', 'SH도시개발공사', 'SH 공사',
       '가양동 현대연합주택조합', '화이트코리아(주식회사)', '신명건설,신원건설,미주건설', 'TEC건설',
   

In [32]:
data["k-시행사"] = data["k-시행사"].fillna('unknown')

data["k-시행사"] = data["k-시행사"].replace("모름", "unknown")
data["k-시행사"] = data["k-시행사"].replace(".", "unknown")
data["k-시행사"] = data["k-시행사"].replace("-", "unknown")
data["k-시행사"] = data["k-시행사"].replace("..", "unknown")
data["k-시행사"] = data["k-시행사"].replace("ㅎㅎㅎ", "unknown")

synonyms = {
    'SH공사': ['sh', 'sh공사', 'sh 공사', '에스에이치공사', '서울주택도시공사', '서울도시주택공사', '에스에치', '에스에치공사', '서울특별시 SH공사', 'SH', 'SH공사', 'SH 공사', 'SH도시개발공사', '서울도시개발공사', '서울시 도시개발공사', '도시개발 공사', '도시개발공사'],
    'LIG건설': ['LIG', 'LIG건설(주)'],
    '대한주택공사': ['대한주택보증(주)', '대한주택보증', '주택공사'],
    '현대건설': ['현대건설(주)', '현대산업개발', '현대', '(주)현대건설'],
    '대우건설': ['대우건설(주)', '남광토건, 대우', '대우,럭키개발,롯데건설,선경건설,태평양건설', '(주)대우건설'],
    '삼성물산': ['삼성물산 건설부문', '삼성건설', '삼성래미안', '삼성관련'],
    '포스코건설': ['포스코건설', '포스코'],
    '롯데건설': ['롯데건설(주)', '롯데건설,우성건설'],
    '한국토지주택공사': ['LH공사', '한국토지주택공사(LH)', 'LH강남지역본부'],
    '금호산업': ['금호산업(주)', '금호건설', '금호', '(주)금호산업'],
    '코오롱건설': ['코오롱건설(주)', '코오롱'],
    '대림산업': ['대림산업', '(주)대림산업'],
    '한화건설': ['한화건설(주)', '(주)한화건설'],
    '한신공영': ['한신공영(주)', '한신'],
    '두산건설': ['두산건설', '두산사우주택조합'],
    'GS건설': ['GS건설주식회사', 'GS 건설'],
    '흥화공업': ['흥화공업(주)', '흥화공업'],
    '삼호건설': ['삼호', '(주)삼호건설', '(주)삼호'],
    '신도종합건설': ['(주)신도종합건설', '신도건설'],
    '동부건설': ['동부건설(주)', '동부건설'],
    '서울시': ['서울시청'],
    '성원건설': ['성원'],
    '신동아건설': ['신동아건설(주)'],
    '우성건설': ['우성', '(주)우성건설'],
    '재건축조합': ['재건축 조합', '조합', '조합아파트'],
    '코리아신탁': ['주)코리아신탁', '코리아신탁주식회사'],
    '한국토지신탁': ['주식회사한국토지신탁'],
    '중앙하이츠': ['중앙건설'],
    '풍림산업': ['풍림산업(주)'],
    'LH' : ['한국토지주택공사'],
    '한보건설': ['한보'],
    '효성건설': ['효성중공업', '(주)효성', '(주)효성건설'],
    '디케이건설': ['(주)디케이건설'],
    '대한토지신탁': ['(주)대한토지신탁'],
    '한국자산신탁': ['한국자산신탁(주)'],
    '동양': ['동양메이저건설'],
    '한양건설': ['(주)한양건설'],
    '(주)제이피홀딩스': ['(주)제이피홀딩스피에프브이'],
    '상부종합건설' : ['상부종합건설(주)']
}

def replace_synonyms(value, synonyms_dict):
    for key, values in synonyms_dict.items():
        if value in values:
            return key
    return value

data['k-시행사'] = data['k-시행사'].apply(lambda x: replace_synonyms(x, synonyms))

company_list = sorted(data["k-시행사"].unique())
print(len(company_list))
for c in company_list:
    print(c)

481
 성사2차 재건축조합
(주)갑조건설
(주)넥서스건설
(주)다보종합건축사
(주)대상
(주)대원종합건설
(주)동아건설산업
(주)디씨엠하우징
(주)리얼티투자개발
(주)목동성원 재건축주택조합
(주)미원
(주)삼전건설
(주)삼종지위드건설
(주)수경하우징
(주)순영종합건설
(주)신한건설
(주)에스떠블유산업개발
(주)엠피씨라이프
(주)영수개발 직장주택조합외 6개 조합
(주)우리건설
(주)제이피홀딩스
(주)중앙건설
(주)청학산업
(주)태영
(주)티에스주택
(주)포유
(주)프리즘지앤시
(주)피플앤하우스
(주)한산디엠씨
(주)해창
(주)현진종합건설
(주)효진테크
BYC
CIT산업개발(주)
GS건설
JR건설
KB신탁사
LG건설
LH
LIG건설
MBC외 6개 직장조합
SH공사
TEC건설
TEC건설+새천년종합건설
unknown
가락시영아파트주택재건축정비사업조합
가양동 현대연합주택조합
가재울뉴타운6구역주택재개발정비사업조합
가재울뉴타운제3구역재재발조합
갈현아파트재건축조합
강남상가 재건축조합
강서재건축조합
강호연립재건조합
개나리6차아파트주택재건축정비사업조합
개나리아파트 주택조합
개포시영아파트 주택재건축 정비사업조합
개포우성9차모델링조합
개포주공2단지주택재건축정비사업조합
건영아파트
건영종합건설, 삼익주택,라이프주택
건우/수정재건축조합
건우3차아파트정비사업조합
건우예원연립재건축주택조합
건정종합건축사사무소
건축조합
경남기업
경동건설
경복아파트주택재건축정비사업조합
경수재건축조합
계룡건설
고덕7단지 주택재건축정비사업조합
고덕주공4단지주택재건축정비사업조합
고려산업개발(주)
고산공영
공군,유진 재건축주택조합
공무원연금관리공단
공영토건
관성아파트재건축조합
광장11차현대지역주택조합
구로3구역재개발조합
구로6구역재개발조합
구로연합주택조합
국방부삼일주택,신도림지역주택조합
국제약품
국제자산신탁(주)
국화연립재건축조합
국회 조합
군인공제조합
군인공제회
극동건설
금하건설
금호17구역재개발조합
금호20주택재개발조합
금호산업
금호아파트재건축조합
금호자이
금호제12구역 재개발조합
금호제13구역 주택재개발정비사업

## 4-5.경비비관리형태, 세대전기계약방법, 청소비관리형태

In [33]:
print(data["경비비관리형태"].unique())

['직영' '위탁' '위탁+직영' nan '기타']


In [34]:
data["경비비관리형태"] = data["경비비관리형태"].fillna('unknown')
data["경비비관리형태"] = data["경비비관리형태"].replace("기타", "unknown")
print(data["경비비관리형태"].value_counts())
print(data["경비비관리형태"].unique())

경비비관리형태
unknown    879378
위탁         208757
직영          33791
위탁+직영        6168
Name: count, dtype: int64
['직영' '위탁' '위탁+직영' 'unknown']


In [35]:
print(data["세대전기계약방법"].unique())

['단일계약' '종합계약' nan]


In [36]:
data["세대전기계약방법"] = data["세대전기계약방법"].fillna('unknown')
print(data["세대전기계약방법"].value_counts())

세대전기계약방법
unknown    885389
종합계약       124610
단일계약       118095
Name: count, dtype: int64


In [37]:
print(data["청소비관리형태"].unique())

['직영' '위탁' '위탁+직영' nan '기타']


In [38]:
data["청소비관리형태"] = data["청소비관리형태"].fillna('unknown')
data["청소비관리형태"] = data["청소비관리형태"].replace("기타", "unknown")
print(data["청소비관리형태"].value_counts())
print(data["청소비관리형태"].unique())

청소비관리형태
unknown    881230
위탁         227511
직영          15174
위탁+직영        4179
Name: count, dtype: int64
['직영' '위탁' '위탁+직영' 'unknown']


## 4-6.아파트명 데이터 정리

- 숫자로만 구성되어 있거나 아파트명이 없는 경우 숫자-숫자 형태로 채워진 데이터가 존재.
- 201호 미공시와 같은 의미없는 값도 존재.
- nan
- 이들을 모두 unknown으로 처리

In [39]:
data.columns

Index(['본번', '부번', '아파트명', '전용면적', '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자',
       '거래유형', '부동산유형', '분양형태', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수',
       'k-전체세대수', 'k-건설사', 'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적',
       'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)',
       'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-등록일자', 'k-수정일자', '경비비관리형태', '세대전기계약방법',
       '청소비관리형태', '건축면적', '주차대수', '단지승인일', '좌표X', '좌표Y', '단지신청일', 'target',
       'is_test', '구', '동', '계약날짜'],
      dtype='object')

In [40]:
# names_dict = {}
# for i, (index, row) in enumerate(data.iterrows()):
#     gu, dong, road_name =  row['구'], row['동'], row['도로명']
#     build_name = row['아파트명']

#     gu_road_name = f"{gu} {road_name}"
#     if not gu_road_name in names_dict:
#         names_dict[f"{gu} {road_name}"] = build_name
    
# print(len(names_dict))
# with open("./building_names.txt", "w") as f:
#     for gr_name, name in names_dict.items():
#         f.write(f"{gr_name} {name}\n")

In [41]:
def convert_to_unknown(x):
    if pd.isna(x):
        return 'unknown'
    if not isinstance(x, str):
        return 'unknown'
    if x.isdigit():
        return 'unknown'
    if '-' in x and all(part.isdigit() for part in x.split('-')):
        return 'unknown'
    return x

data['아파트명'] = data['아파트명'].apply(convert_to_unknown)
data['아파트명'] = data['아파트명'].replace('201호 미공시', 'unknown')

In [42]:
with open("./unique_building_names.txt", "w") as f:
    for name in sorted(data['아파트명'].unique()):
        f.write(f"{name}\n")

# 5.처리된 데이터 분할 및 저장

In [43]:
train_df = data[data['is_test'] == 0]
test_df = data[data['is_test'] == 1]

train_df = train_df.drop(columns=['is_test'])
test_df = test_df.drop(columns=['is_test', 'target'])

train_df.to_csv(f'{data_path}/new/new_train_ver3.csv', index=False)
test_df.to_csv(f'{data_path}/new/new_test_ver3.csv', index=False)

In [44]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1118822 entries, 0 to 1118821
Data columns (total 42 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   본번                      1118822 non-null  float64       
 1   부번                      1118822 non-null  float64       
 2   아파트명                    1118822 non-null  object        
 3   전용면적                    1118822 non-null  float64       
 4   층                       1118822 non-null  int64         
 5   건축년도                    1118822 non-null  int64         
 6   도로명                     1118822 non-null  object        
 7   해제사유발생일                 5983 non-null     float64       
 8   등기신청일자                  1118822 non-null  object        
 9   거래유형                    1118822 non-null  object        
 10  부동산유형                   1118822 non-null  object        
 11  분양형태                    1118822 non-null  object        
 12  k-관리방식             

In [45]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9272 entries, 0 to 9271
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   본번                      9272 non-null   float64       
 1   부번                      9272 non-null   float64       
 2   아파트명                    9272 non-null   object        
 3   전용면적                    9272 non-null   float64       
 4   층                       9272 non-null   int64         
 5   건축년도                    9272 non-null   int64         
 6   도로명                     9272 non-null   object        
 7   해제사유발생일                 212 non-null    float64       
 8   등기신청일자                  9272 non-null   object        
 9   거래유형                    9272 non-null   object        
 10  부동산유형                   9272 non-null   object        
 11  분양형태                    9272 non-null   object        
 12  k-관리방식                  9272 non-null   object       